In [2]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [3]:
#from google.colab import files
import os
#import tensorflow as tf
#assert tf.__version__.startswith('2')
from mediapipe_model_maker import gesture_recognizer

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
dataset_zip = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ColabNotebooks', 'gestures_dataset.zip')
!unzip {dataset_zip}
dataset_path = "gestures_dataset"

Streaminguitvoer ingekort tot de laatste 5000 regels.
  inflating: gestures_dataset/peace/2968e89f-26fd-4347-90c7-1490e6a61749.jpg  
  inflating: gestures_dataset/peace/29939b11-8cfe-42da-93a7-82df4dd75cdc.jpg  
  inflating: gestures_dataset/peace/29b8567a-346f-4dc7-90c8-454daf945df0.jpg  
  inflating: gestures_dataset/peace/29daf45b-ad97-40bf-b4e1-112c8f791ad5.jpg  
  inflating: gestures_dataset/peace/29fc7a5b-968a-4d62-b115-ecc7daeafeb3.jpg  
  inflating: gestures_dataset/peace/2a18c359-e260-4adb-a1b8-b4631977cf65.jpg  
  inflating: gestures_dataset/peace/2a3a1bf7-d019-4aef-8966-f2096b98bd04.jpg  
  inflating: gestures_dataset/peace/2a6f830e-74f3-4bbd-acc8-aa6b5aae853d.jpg  
  inflating: gestures_dataset/peace/2a8db779-0237-4c0c-a0ad-7b1ed7f409f3.jpg  
  inflating: gestures_dataset/peace/2ab0dec4-192a-4b25-8e4a-b1d74c1e129f.jpg  
  inflating: gestures_dataset/peace/2ad33232-33ed-4a27-baa6-e5180b8e8ca4.jpg  
  inflating: gestures_dataset/peace/2aecad17-acc3-4b50-a0b9-72e7f839756f.jpg 

In [5]:
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    number_images = len(os.listdir(os.path.join(dataset_path, i)))
    labels.append((i, number_images))
print(labels)

[('open_palm', 1770), ('peace', 1769), ('surf', 1763), ('none', 1600), ('three', 1751)]


In [6]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [7]:
model_name = "custom_gesture_recognizer"
hparams = gesture_recognizer.HParams(export_dir=model_name)
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 5)                 645       
 out (Dense)                                                     
                                                             

In [8]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

750/750 [==============================] - 4s 2ms/step - loss: 0.2425 - categorical_accuracy: 0.8827
Test loss:0.24254289269447327, Test accuracy:0.8826666474342346


In [9]:
model.export_model()

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite


In [10]:
!zip -r custom_gesture_recognizer.zip /content/custom_gesture_recognizer

  adding: content/custom_gesture_recognizer/ (stored 0%)
  adding: content/custom_gesture_recognizer/metadata.json (deflated 69%)
  adding: content/custom_gesture_recognizer/epoch_models/ (stored 0%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0005.index (deflated 55%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0002.data-00000-of-00001 (deflated 15%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0001.index (deflated 55%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0004.data-00000-of-00001 (deflated 15%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0008.index (deflated 55%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0010.index (deflated 55%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0006.data-00000-of-00001 (deflated 15%)
  adding: content/custom_gesture_recognizer/epoch_models/model-0005.data-00000-of-00001 (deflated 15%)
  adding: content/custom_gesture